##### Import

In [37]:
import rx
from rx import operators as ops
from urllib3 import Retry
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS, ASYNCHRONOUS
from collections import OrderedDict
from csv import DictReader
from os import getcwd, listdir
from os.path import join, exists, isfile, isdir, abspath, pardir
from time import time

##### Configs

In [38]:
# You can generate an API token from the "API Tokens Tab" in the UI
token = "RtWgHEkMiipthr5IkT1RpRed4ZXRH-9Q8YuEzZ3pdkjgelVA83D1SZ5tF9V_NY3u7W5-lAl-GuVUC3n3mS1cCQ=="
org = "ulb"
bucket = "advanced_db"
port=8086
url="http://localhost:{}".format(port)

In [39]:
# file_name = "trades_btc_1_day.csv"
file_path = join(abspath(join(getcwd(), pardir)), "data", "data_trades")
files = listdir(file_path)

In [40]:
files = [x for x in files if isfile(join(file_path, x)) and str(x).endswith("_day.csv")]
sorted(files)

['trades_btc_14_day.csv',
 'trades_btc_1_day.csv',
 'trades_btc_2_day.csv',
 'trades_btc_30_day.csv',
 'trades_btc_3_day.csv',
 'trades_btc_4_day.csv',
 'trades_btc_5_day.csv',
 'trades_btc_6_day.csv',
 'trades_btc_7_day.csv',
 'trades_btc_90_day.csv',
 'trades_eth_14_day.csv',
 'trades_eth_1_day.csv',
 'trades_eth_2_day.csv',
 'trades_eth_30_day.csv',
 'trades_eth_3_day.csv',
 'trades_eth_4_day.csv',
 'trades_eth_5_day.csv',
 'trades_eth_6_day.csv',
 'trades_eth_7_day.csv',
 'trades_eth_90_day.csv']

In [41]:
def parse_trade_row(row: OrderedDict):
    return Point("trades").tag("symbol", "{}".format(row['symbol'])) \
        .field("side", 1 if row['side'] == 'buy' else -1) \
        .field("price", float(row['price'])) \
        .field("amount", float(row['amount'])) \
        .time(datetime.fromtimestamp(float(row['time_stamp'])))

In [42]:
for file in files:
    data = list(map(parse_trade_row, DictReader(open(join(file_path, file), 'r'))))
    with InfluxDBClient(url=url, token=token, org=org) as client:
        with client.write_api(write_options=ASYNCHRONOUS) as write_api:
            start_time = time()
            write_api.write(bucket=bucket, record=data)
            print("Took {} seconds to load {} records in {}".format( time() - start_time, len(data), file))

Took 3.677189826965332 seconds to load 345608 records in trades_btc_1_day.csv
Took 18.22838592529297 seconds to load 1728001 records in trades_eth_5_day.csv
Took 115.78496789932251 seconds to load 10368201 records in trades_btc_30_day.csv
Took 382.1422460079193 seconds to load 30922328 records in trades_btc_90_day.csv
Took 14.578586101531982 seconds to load 1382399 records in trades_eth_4_day.csv
Took 21.900441884994507 seconds to load 2073599 records in trades_eth_6_day.csv
Took 7.31665301322937 seconds to load 691215 records in trades_btc_2_day.csv
Took 51.98059582710266 seconds to load 4838496 records in trades_btc_14_day.csv
Took 26.10172986984253 seconds to load 2419200 records in trades_eth_7_day.csv
Took 11.113489866256714 seconds to load 1036820 records in trades_btc_3_day.csv
Took 55.19174909591675 seconds to load 4838401 records in trades_eth_14_day.csv
Took 22.41773009300232 seconds to load 2073640 records in trades_btc_6_day.csv
Took 7.507562160491943 seconds to load 691200

In [8]:
data = list(map(parse_trade_row, DictReader(open(file_path, 'r'))))

In [17]:
with InfluxDBClient(url=url, token=token, org=org) as client:
    # with client.write_api(write_options=WriteOptions(batch_size=50_000, flush_interval=10_000)) as write_api:
    with client.write_api(write_options=ASYNCHRONOUS) as write_api:
        start_time = time()
        write_api.write(bucket=bucket, record=data)
        print("Took {} seconds to load {} records in {}".format( time() - start_time, len(data), file_name))

Took 3.7934329509735107 seconds to load 345608 records in trades_btc_1_day.csv


In [19]:
_data_ = list()

for index, row in enumerate(DictReader(open(file_path, 'r'))):
    # point = parse_trade_row(row)
    print(datetime.fromtimestamp(float(row['time_stamp'])))
    # print("Rows: {}".format(point))
    if index == 1:
        break

2021-11-20 01:00:00.168152
2021-11-20 01:00:00.418139


In [12]:
client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

p = Point("city_temp").tag("location", "brussels").field("temperature", 25.3)

write_api.write(bucket="demo", record=data)

## using Table structure
tables = query_api.query('from(bucket:"demo") |> range(start: -10m)')

In [30]:
data

In [29]:
write_api.write(bucket="demo", record=data)

In [15]:
for table in tables:
    print(table)
    for row in table.records:
        print (row.values)


# ## using csv library
# csv_result = query_api.query_csv('from(bucket:"my-bucket") |> range(start: -10m)')
# val_count = 0
# for row in csv_result:
#     for cell in row:
#         val_count += 1

FluxTable() columns: 9, records: 1
{'result': '_result', 'table': 0, '_start': datetime.datetime(2021, 11, 21, 19, 22, 43, 356533, tzinfo=tzutc()), '_stop': datetime.datetime(2021, 11, 21, 19, 32, 43, 356533, tzinfo=tzutc()), '_time': datetime.datetime(2021, 11, 21, 19, 32, 42, 783335, tzinfo=tzutc()), '_value': 25.3, '_field': 'temperature', '_measurement': 'city_temp', 'location': 'brussels'}


In [16]:
csv_result = query_api.query_csv('from(bucket:"demo") |> range(start: -10m)')
val_count = 0
for row in csv_result:
    for cell in row:
        val_count += 1

In [28]:
tables[0].get_group_key()

[FluxColumn(2, label='_start', data_type='dateTime:RFC3339', group=True, default_value=''),
 FluxColumn(3, label='_stop', data_type='dateTime:RFC3339', group=True, default_value=''),
 FluxColumn(6, label='_field', data_type='string', group=True, default_value=''),
 FluxColumn(7, label='_measurement', data_type='string', group=True, default_value=''),
 FluxColumn(8, label='location', data_type='string', group=True, default_value='')]

In [23]:
query_api = client.query_api()

In [15]:
query = 'from(bucket:"advanced_db")' \
        ' |> range(start: 0, stop: now())' \
        ' |> filter(fn: (r) => r._measurement == "trades")'
result = client.query_api().query(query=query)

In [16]:
result

[]